In [11]:
import pandas as pd
import csv
import requests
import numpy as np
import json
import plotly.express as px
import plotly.io as pio


#Download_csv_file

CSV_URL = "https://api.covid19india.org/csv/latest/districts.csv"
with requests.Session() as s:
    download = s.get(CSV_URL)

    
#Read and write to csv

with open('covid19india_districts.csv', 'w') as f:
    writer = csv.writer(f)
    for line in download.iter_lines():
        writer.writerow(line.decode('utf-8').split(','))


#Creating Pivot table

districts=pd.read_csv('covid19india_districts.csv')
table = pd.pivot_table(districts, values='Confirmed', index=['District','State'], columns=['Date'], fill_value=0)
table.to_csv("district_pivot_confirmed.tsv", sep='\t')
pivot_table_confirmed=pd.read_csv("district_pivot_confirmed.tsv", sep="\t")

table = pd.pivot_table(districts, values='Tested', index=['District','State'], columns=['Date'], fill_value=0)
table.to_csv("district_pivot_tested.tsv", sep='\t')
pivot_table_tested=pd.read_csv("district_pivot_tested.tsv", sep="\t")

#Creating two weeks data

last_week_confirmed = pivot_table_confirmed.T.tail(7).T
last_week_confirmed['total_confirmed'] = last_week_confirmed.sum(axis=1)

last_week_tested = pivot_table_tested.T.tail(7).T
last_week_tested['total_tested'] = last_week_tested.sum(axis=1)

first_column_confirmed = pivot_table_confirmed.T.head(2).T
first_column_tested = pivot_table_tested.T.head(2).T

confirmed = first_column_confirmed.join(last_week_confirmed)
tested = first_column_tested.join(last_week_tested)


merged = confirmed.merge(tested, on=['District','State'])
merged['PositivityRate'] = merged['total_confirmed']/merged['total_tested']


searchfor=['Capital Complex','Delhi','Gaurela Pendra Marwahi','Other State']
merged2 = merged[~merged.District.str.contains('|'.join(searchfor))]


header=['District','State','total_confirmed','total_tested','PositivityRate']
merged2.to_csv("district_positivity.tsv", sep='\t',columns=header)

#Creating image
pio.renderers.default = 'browser'

india_districts = json.load(open("india_districts.geojson", "r"))

district_id_map = {}
for feature in india_districts["features"]:
    feature["id"] = feature["properties"]["dt_code"]
    district_id_map[feature["properties"]["district"]] = feature["id"]


df = pd.read_csv("district_positivity.tsv",sep='\t')
df["id"] = df["District"].apply(lambda x: district_id_map[x])
#df.loc[df['Rate_of_Increase'] <= 0, 'Increase_image'] = 0
#df.loc[df['Rate_of_Increase'] > 0, 'Increase_image'] = df['Rate_of_Increase']



fig = px.choropleth(
    df,
    locations="id",
    geojson=india_districts,
    color="PositivityRate",
    hover_name="District",
    hover_data=["PositivityRate"],
    title="COVID19 Positivity rate per week",
    color_continuous_scale=px.colors.diverging.BrBG,
    color_continuous_midpoint=0)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [17]:
df.sort_values(by=['PositivityRate'], inplace=True, ascending=False, ignore_index=True)
df.loc[0:9, ['District','State', 'PositivityRate']]

,District,State,PositivityRate
0,Nuapada,Odisha,72.870046
1,Rayagada,Odisha,32.520761
2,Boudh,Odisha,24.461694
3,Ahmednagar,Maharashtra,24.086885
4,Durg,Chhattisgarh,19.148520
5,Nayagarh,Odisha,18.067935
6,Dhamtari,Chhattisgarh,17.915770
7,Kalahandi,Odisha,16.282677
8,Surajpur,Chhattisgarh,15.733646
9,Sambalpur,Odisha,14.776069
